# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import random

from extrucal.extrusion import throughput_cal

from sklearn import datasets
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.dummy import DummyClassifier, DummyRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.model_selection import (
    GridSearchCV,
    RandomizedSearchCV,
    cross_val_score,
    cross_validate,
    train_test_split,
)
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
import warnings

from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm.sklearn import LGBMClassifier, LGBMRegressor
from xgboost import XGBClassifier, XGBRegressor

from sklearn.utils import shuffle

from sklearn.naive_bayes import MultinomialNB

from tqdm import tqdm

random.seed(0)

# Dataset Read In

In [2]:
df = pd.read_csv("../data/extrucal_dataset_improved.csv")
df

,extruder_size,metering_depth,polymer_density,rpm,screw_pitch,flight_width,number_flight,throughput
0,80,7.2,1200,95,48.0,12.0,2,92.55
1,220,17.6,1300,75,154.0,13.2,1,4397.68
2,190,7.6,1400,95,323.0,36.1,2,3187.76
3,190,7.6,800,55,228.0,26.6,1,1008.12
4,60,1.8,800,10,48.0,10.8,1,2.72
...,...,...,...,...,...,...,...,...
1935355,40,3.2,1000,30,32.0,4.0,1,8.78
1935356,180,3.6,800,85,306.0,27.0,2,762.58
1935357,30,1.8,1100,60,30.0,2.7,1,7.89
1935358,70,6.3,1000,40,35.0,10.5,2,11.59


# Useful Functions

In [3]:
def mean_std_cross_val_scores(model, X_train, y_train, **kwargs):
    """
    Returns mean and std of cross validation

    Parameters
    ----------
    model :
        scikit-learn model
    X_train : numpy array or pandas DataFrame
        X in the training data
    y_train :
        y in the training data

    Returns
    ----------
        pandas Series with mean scores from cross_validation
    """

    scores = cross_validate(model, X_train, y_train, **kwargs)

    mean_scores = pd.DataFrame(scores).mean()
    std_scores = pd.DataFrame(scores).std()
    out_col = []

    for i in range(len(mean_scores)):
        out_col.append((f"%0.3f (+/- %0.3f)" % (mean_scores[i], std_scores[i])))

    return pd.Series(data=out_col, index=mean_scores.index)

In [4]:
def mape(true, pred):
    return 100.0 * np.mean(np.abs((pred - true) / (true+0.1)))  # 0.1 was added to prevent division by zero

# make a scorer function that we can pass into cross-validation
mape_scorer = make_scorer(mape, greater_is_better=False)

# Train/Test Split

In [5]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=123, shuffle=True)
train_df.head()

,extruder_size,metering_depth,polymer_density,rpm,screw_pitch,flight_width,number_flight,throughput
665324,120,9.6,1400,50,72.0,24.0,2,93.53
775241,230,9.2,1200,45,161.0,43.7,2,624.11
867365,220,11.0,800,90,242.0,13.2,1,3208.52
1458314,250,10.0,1150,0,300.0,50.0,2,0.00
1097383,70,5.6,800,45,77.0,7.0,2,64.95


In [6]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1548288 entries, 665324 to 773630
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   extruder_size    1548288 non-null  int64  
 1   metering_depth   1548288 non-null  float64
 2   polymer_density  1548288 non-null  int64  
 3   rpm              1548288 non-null  int64  
 4   screw_pitch      1548288 non-null  float64
 5   flight_width     1548288 non-null  float64
 6   number_flight    1548288 non-null  int64  
 7   throughput       1548288 non-null  float64
dtypes: float64(4), int64(4)
memory usage: 106.3 MB


In [7]:
X_train = train_df.drop(columns=["throughput"])
y_train = train_df["throughput"]

X_test = test_df.drop(columns=["throughput"])
y_test = test_df["throughput"]

# Evaluation of ML Models

In [8]:
results = {}

### 1. Baseline Model

In [9]:
pipe_dummy = make_pipeline(StandardScaler(), DummyRegressor())

In [10]:
results["Dummy"] = mean_std_cross_val_scores(
    pipe_dummy, X_train, y_train, return_train_score=True, scoring=mape_scorer
)

In [11]:
pd.DataFrame(results)

,Dummy
fit_time,0.289 (+/- 0.036)
score_time,0.024 (+/- 0.001)
test_score,-54716.817 (+/- 255.159)
train_score,-54716.790 (+/- 62.364)


### 2. ML Models

In [12]:
random_state = 123

pipe_ridge = make_pipeline(StandardScaler(), Ridge(max_iter=2000, random_state=random_state))
pipe_lasso = make_pipeline(StandardScaler(), Lasso(max_iter=2000, random_state=random_state))
pipe_rf = make_pipeline(StandardScaler(), RandomForestRegressor(random_state=random_state))
pipe_xgb = make_pipeline(StandardScaler(), XGBRegressor(random_state=random_state, verbosity=0))
pipe_lgbm = make_pipeline(StandardScaler(), LGBMRegressor(random_state=random_state))
pipe_catb = make_pipeline(StandardScaler(), CatBoostRegressor(random_state=random_state, verbose=0, loss_function='RMSE'))

In [13]:
models = {
    "ridge": pipe_ridge,
    "lass": pipe_lasso,
    "random_forest": pipe_rf,
    "XGBoost": pipe_xgb,
    "LightGBM": pipe_lgbm,
    "CatBoost": pipe_catb
}

In [14]:
%%time

for (name, model) in tqdm(models.items()):
    results[name] = mean_std_cross_val_scores(
        model, X_train, y_train, return_train_score=True, scoring=mape_scorer
    )

100%|█████████████████████████████████████████████████████████████████████████████████| 6/6 [1:18:39<00:00, 786.56s/it]

CPU times: total: 2h 28min 29s
Wall time: 1h 18min 39s


In [15]:
pd.DataFrame(results)

,Dummy,ridge,lass,random_forest,XGBoost,LightGBM,CatBoost
fit_time,0.289 (+/- 0.036),0.392 (+/- 0.031),0.979 (+/- 0.076),495.887 (+/- 4.003),66.118 (+/- 8.670),6.884 (+/- 0.647),174.947 (+/- 1.844)
score_time,0.024 (+/- 0.001),0.033 (+/- 0.004),0.038 (+/- 0.005),78.808 (+/- 4.474),0.250 (+/- 0.061),0.398 (+/- 0.018),0.120 (+/- 0.025)
test_score,-54716.817 (+/- 255.159),-56174.033 (+/- 274.268),-56105.541 (+/- 274.140),-3.939 (+/- 0.014),-1453.535 (+/- 79.408),-1728.126 (+/- 12.210),-52.496 (+/- 0.778)
train_score,-54716.790 (+/- 62.364),-56174.357 (+/- 57.429),-56105.882 (+/- 57.217),-1.502 (+/- 0.002),-1437.271 (+/- 75.226),-1729.708 (+/- 12.728),-52.488 (+/- 0.869)
